In [ ]:
!pip install langchain chromadb faiss-cpu openai tiktoken langchain_openai langchain-community wikipedia

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 68.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 89.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.8/84.8 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 79.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 82.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 52.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 65.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━

Wikipedia Retriver

In [ ]:
from langchain_community.retrievers import WikipediaRetriever

In [ ]:
#Initialize the retrever(optinal:set lanuage and top_K)
retriever=WikipediaRetriever(top_k=1,lang='en')

In [ ]:
#Deefine Query
query="Leonardo Da Vinci "
#Get relevaent Wikipedia documents
docs=retriever.invoke(query)

In [ ]:
docs

[Document(metadata={'title': 'Leonardo da Vinci', 'summary': "Leonardo di ser Piero da Vinci (15 April 1452 – 2 May 1519) was an Italian polymath of the High Renaissance who was active as a painter, draughtsman, engineer, scientist, theorist, sculptor, and architect. While his fame initially rested on his achievements as a painter, he has also become known for his notebooks, in which he made drawings and notes on a variety of subjects, including anatomy, astronomy, botany, cartography, painting, and palaeontology. Leonardo is widely regarded as a genius who epitomised the Renaissance humanist ideal, and his collective works contributed to the development of European art to an extent rivalled only by that of his younger contemporary Michelangelo.\nBorn out of wedlock to a successful notary and a lower-class woman in, or near, Vinci, he was educated in Florence by the Italian painter and sculptor Andrea del Verrocchio. He began his career in the city, but then spent much time in the serv

In [ ]:
for i , doc in enumerate(docs):
  print(f"\n--Result{i+1}--")
  print(f"Content:\n{doc.page_content}.......")


--Result1--
Content:
Leonardo di ser Piero da Vinci (15 April 1452 – 2 May 1519) was an Italian polymath of the High Renaissance who was active as a painter, draughtsman, engineer, scientist, theorist, sculptor, and architect. While his fame initially rested on his achievements as a painter, he has also become known for his notebooks, in which he made drawings and notes on a variety of subjects, including anatomy, astronomy, botany, cartography, painting, and palaeontology. Leonardo is widely regarded as a genius who epitomised the Renaissance humanist ideal, and his collective works contributed to the development of European art to an extent rivalled only by that of his younger contemporary Michelangelo.
Born out of wedlock to a successful notary and a lower-class woman in, or near, Vinci, he was educated in Florence by the Italian painter and sculptor Andrea del Verrocchio. He began his career in the city, but then spent much time in the service of Ludovico Sforza in Milan. Later, h

Vector Store Retriever

In [ ]:
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_core.documents import Document

In [ ]:
#Step1 :Document Source
documents=[
    Document(page_content="A bunch of scientists bring back dinosaurs and mayhem breaks loose"),
    Document(page_content="Chroma is a vector Database optimized for LLM-based search."),
    Document(page_content="Embeddings convert text into high-dimensional vectors."),
    Document(page_content="OpenAI provides powerful embedding model")
]

In [ ]:
#Step2: Initialize Embedding model
embedding_model=OpenAIEmbeddings()

#Step3 : Create Chroma vector store in memory
vectorstore =Chroma.from_documents(
    documents=documents,
    embedding=embedding_model,
    collection_name="my_collection"
)

In [ ]:
#Step4 :Convert vectorstore into a retriever
retriever=vectorstore.as_retriever(search_kwargs={"k:2"})


In [ ]:
query="What is Chroma used for?"
results=retriever.invoke(query)

In [ ]:
for i , doc in enumerate(results):
  print(f"\n--Result{i+1}--")
  print(f"Content:\n{doc.page_content}")

Maximal Marginal Relevance(MMR)

In [ ]:
#Step1 :Document Source
docs=[
    Document(page_content="A bunch of scientists bring back dinosaurs and mayhem breaks loose"),
    Document(page_content="Chroma is a vector Database optimized for LLM-based search."),
    Document(page_content="Embeddings convert text into high-dimensional vectors."),
    Document(page_content="OpenAI provides powerful embedding model")
]

In [ ]:
from langchain_community.vectorstores import FAISS

embedding_model = OpenAIEmbeddings()

vectorstore = FAISS.from_documents(documents=docs, embedding=embedding_model)

In [ ]:
retriever=vectorstore.as_retriever(
    search_type='mmr',
    search_kwargs={"k":3,"lambda_mult":1}
)

In [ ]:
query="What is Chroma?"
results=retriever.invoke(query)

In [ ]:
for i , doc in enumerate(results):
  print(f"\n--Result{i+1}--")
  print(f"Content:\n{doc.page_content}")

Multi-Query-Retriver

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_core.documents import Document
from langchain_openai import ChatOpenAI
from langchain.retrievers.multi_query import MultiQueryRetriever



ModuleNotFoundError: No module named 'langchain.retrievers'

In [ ]:
from langchain.schema import Document

documents = [
    Document(
        page_content="""
        MultiQueryRetriever enhances document retrieval by generating
        multiple alternative queries from a single user question.
        This helps retrieve relevant documents that use different phrasing.
        """,
        metadata={"source": "doc1"}
    ),
    Document(
        page_content="""
        Query expansion is a retrieval strategy where an LLM rewrites
        the original question into several semantically similar versions
        to improve recall in vector search.
        """,
        metadata={"source": "doc2"}
    ),
    Document(
        page_content="""
        Single-query similarity search can fail when documents use
        synonyms or indirect wording. Multi-query retrieval reduces
        this risk by searching with multiple reformulated queries.
        """,
        metadata={"source": "doc3"}
    ),
    Document(
        page_content="""
        In RAG systems, MultiQueryRetriever is often used to overcome
        limitations of embedding-based similarity search by broadening
        the search space using LLM-generated queries.
        """,
        metadata={"source": "doc4"}
    ),
    Document(
        page_content="""
        Large language models can paraphrase a user question in many ways.
        These paraphrases can be used as separate search queries to find
        documents that would otherwise be missed.
        """,
        metadata={"source": "doc5"}
    ),
    Document(
        page_content="""
        Vector databases retrieve documents based on semantic similarity,
        but exact phrasing still matters. Multi-query approaches help
        match documents with different vocabulary choices.
        """,
        metadata={"source": "doc6"}
    ),
    Document(
        page_content="""
        Multi-query retrieval improves recall but may increase latency
        because several searches are performed instead of one.
        This tradeoff is common in high-quality RAG pipelines.
        """,
        metadata={"source": "doc7"}
    ),
    Document(
        page_content="""
        When users ask vague or ambiguous questions, generating multiple
        interpretations of the query can significantly improve retrieval
        quality and downstream answer accuracy.
        """,
        metadata={"source": "doc8"}
    ),
    Document(
        page_content="""
        Query reformulation is especially useful when documents were written
        by different authors who use inconsistent terminology for
        the same underlying concepts.
        """,
        metadata={"source": "doc9"}
    ),
    Document(
        page_content="""
        Compared to basic retrievers, MultiQueryRetriever is better at
        handling natural language variation, making it a strong choice
        for production-grade question answering systems.
        """,
        metadata={"source": "doc10"}
    ),
]


In [ ]:
pip install -U langchain langchain-community langchain-core langchain-openai


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.8/108.8 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.4/157.4 kB 8.3 MB/s eta 0:00:00
  Attempting uninstall: langgraph-prebuilt
    Found existing installation: langgraph-prebuilt 1.0.6
    Uninstalling langgraph-prebuilt-1.0.6:
      Successfully uninstalled langgraph-prebuilt-1.0.6
  Attempting uninstall: langgraph
    Found existing installation: langgraph 1.0.6
    Uninstalling langgraph-1.0.6:
      Successfully uninstalled langgraph-1.0.6
  Attempting uninstall: langchain
    Found existing installation: langchain 1.2.4
    Uninstalling langchain-1.2.4:
      Successfully uninstalled langchain-1.2.4


In [ ]:
#Initialize OpenAI embeddings
from langchain_community.vectorstores import FAISS

embedding_model = OpenAIEmbeddings()

vectorstore = FAISS.from_documents(documents=docs, embedding=embedding_model)

In [ ]:
#Create retrievers
similarity_retriever = vectorstore.as_retriever(search_type='similarity',search_kwargs={'k':2})


In [ ]:
multiquery_retriever=MultiQueryRetriever(
    retriever=similarity_retriever(search_kwargs={"k":5}),
    llm=ChatOpenAI(model_name='gpt-4.0  -turbo',temperature=0)
)

In [ ]:
query="Describe me about llm"


In [ ]:
#Retriever result
similarity_retriever=similarity_retriever.invoke(query)
multiquery_retriever=multiquery_retriever.invoke(query)

In [ ]:
for i , doc in enumerate(similarity_retriever):
  print(f"\n--Similarity Result{i+1}--")
  print(f"Content:\n{doc.page_content}")

for i , doc in enumerate(multiquery_retriever):
  print(f"\n--Multi-Query Result{i+1}--")
  print(f"Content:\n{doc.page_content}")